In [1]:
import pandas as pd
from statsmodels.api import Logit
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm
import pickle

In [2]:
# Importation du jeux de données
data_test = pd.read_csv("billets_production.csv")
data_test

,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5


In [3]:
# Import de notre modèle entrainé
detecteur_fb = pickle.load(open("detecteur_fb.pickle", "rb"))

In [4]:
# pre-precessing
X_log = data_test[['margin_low', 'margin_up', 'length']]
X_log = sm.tools.add_constant(X_log)

# Rajout des colonnes prediction et proba
data_test["proba_genuine"] = detecteur_fb.predict(X_log)
data_test["predicted_genuine"] = (data_test["proba_genuine"] >= 0.5).astype(int)

data_test

,diagonal,height_left,height_right,margin_low,margin_up,length,id,proba_genuine,predicted_genuine
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1,0.000007,0
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2,0.000003,0
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3,0.000028,0
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4,0.999089,1
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5,0.999979,1


In [5]:
# Comptage des vrais et faux billet
data_test.value_counts("predicted_genuine")

predicted_genuine
0    3
1    2
dtype: int64

### Matrice de confusion (si besoin)

mat_conf = confusion_matrix(data_test["is_genuine"], data_test['y_pred'])


sns.heatmap(mat_conf,
            annot = True,
            fmt = ".3g", 
            cmap = sns.color_palette("magma", as_cmap=True),
            linecolor = "white",
            linewidths = 0.3,
            xticklabels = ["Faux","Vrai"],
            yticklabels=["Faux","Vrai"])
plt.xlabel("Prédictions")
plt.ylabel("Valeurs réelles")
plt.title("Matrice de confusion du data_test");